# Synthetic Data Generation

# Deep Learning Approach

## Getting inputs ready

In [2]:
!pip install scipy==1.8.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 17.9 MB/s eta 0:00:00


In [3]:
!pip install torch==2.0.0 -q

In [4]:
!pip install sdv==0.13.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.9/52.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.8/39.8 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
#Importing necessary libraries
import os
import logging
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sdv.evaluation import evaluate
from sdv.metrics.tabular import CSTest, KSTest, LogisticDetection, SVCDetection
from sdv.tabular import CTGAN, TVAE, CopulaGAN, GaussianCopula
import warnings
from ctgan.synthesizers.ctgan import CTGANSynthesizer

warnings.filterwarnings("ignore")

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## With Label Encoding

### Getting the input data which is the actual data as a dataframe

In [7]:
filename = "Consolidated_Actual_Fault_Data.csv"
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/" + filename)
display(df)

,H2,CH4,C2H2,C2H4,C2H6,Fault
0,80.00,10.00,0.00,4.00,1.50,NF
1,46.13,11.57,0.63,33.14,8.52,NF
2,33.66,2.97,2.48,33.17,27.72,NF
3,143.80,21.53,0.20,8.21,2.71,NF
4,170.00,300.00,8.90,580.00,44.00,T3
...,...,...,...,...,...,...
468,165.62,240.95,61.32,514.53,13.53,T3
469,164.00,244.00,103.00,497.00,8.30,T3
470,135.65,278.53,58.86,492.00,2.95,T3
471,135.88,362.42,125.22,826.65,3.74,T3


### Encoding the column Fault using Label encoder

In [8]:
le = LabelEncoder()
df["Fault"] = le.fit_transform(df["Fault"])

In [9]:
df

,H2,CH4,C2H2,C2H4,C2H6,Fault
0,80.00,10.00,0.00,4.00,1.50,2
1,46.13,11.57,0.63,33.14,8.52,2
2,33.66,2.97,2.48,33.17,27.72,2
3,143.80,21.53,0.20,8.21,2.71,2
4,170.00,300.00,8.90,580.00,44.00,7
...,...,...,...,...,...,...
468,165.62,240.95,61.32,514.53,13.53,7
469,164.00,244.00,103.00,497.00,8.30,7
470,135.65,278.53,58.86,492.00,2.95,7
471,135.88,362.42,125.22,826.65,3.74,7


### Synthetic data generation using CTGANSynthesizer

In [47]:
data = df.copy()

In [42]:
data.columns.values

array(['H2', 'CH4', 'C2H2', 'C2H4', 'C2H6', 'Fault'], dtype=object)

In [46]:
discrete_columns = ['H2', 'CH4', 'C2H2', 'C2H4', 'C2H6', 'Fault']

In [48]:
ctgan = CTGANSynthesizer(epochs=200)
ctgan.fit(data, discrete_columns)

In [49]:
samples = ctgan.sample(5000)
samples.head()
print(samples.shape)

(5000, 6)


In [50]:
evaluate(samples, df)

0.7120043092118631

Saving the samples generated into CSV file

In [51]:
samples.to_csv("/content/gdrive/My Drive/Colab Notebooks/Synthetic_Data/LE_CTGANSynth_200.csv", index = False)

## Create new samples using CTGAN

In [52]:
data_aug = df.copy()

In [53]:
data_aug.head()

,H2,CH4,C2H2,C2H4,C2H6,Fault
0,80.00,10.00,0.00,4.00,1.50,2
1,46.13,11.57,0.63,33.14,8.52,2
2,33.66,2.97,2.48,33.17,27.72,2
3,143.80,21.53,0.20,8.21,2.71,2
4,170.00,300.00,8.90,580.00,44.00,7


In [76]:
model = CTGAN()

In [77]:
model.fit(data_aug)

In [78]:
augmented_data = model.sample(5000)
augmented_data.head()

,H2,CH4,C2H2,C2H4,C2H6,Fault
0,0.000,535.73,0.000,0.00,0.00,5
1,5384.799,1497.23,92768.588,0.00,2308.80,7
2,2111.793,693.70,0.000,0.00,3057.59,1
3,1821.057,3148.59,3546.065,0.00,0.00,3
4,1179.670,0.00,0.000,352030.56,2438.51,0


In [79]:
evaluate(augmented_data, df)

0.49276998149846934

In [80]:
samples.to_csv("/content/gdrive/My Drive/Colab Notebooks/Synthetic_Data/LE_CTGAN_Def.csv", index = False)

## Without Label Encoding

### Getting the input data which is the actual data as a dataframe

In [59]:
filename = "Consolidated_Actual_Fault_Data.csv"
df1 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/" + filename)
display(df1)

,H2,CH4,C2H2,C2H4,C2H6,Fault
0,80.00,10.00,0.00,4.00,1.50,NF
1,46.13,11.57,0.63,33.14,8.52,NF
2,33.66,2.97,2.48,33.17,27.72,NF
3,143.80,21.53,0.20,8.21,2.71,NF
4,170.00,300.00,8.90,580.00,44.00,T3
...,...,...,...,...,...,...
468,165.62,240.95,61.32,514.53,13.53,T3
469,164.00,244.00,103.00,497.00,8.30,T3
470,135.65,278.53,58.86,492.00,2.95,T3
471,135.88,362.42,125.22,826.65,3.74,T3


### Synthetic data generation using CTGANSynthesizer

In [60]:
data1 = df1.copy()

In [61]:
data1.columns.values

array(['H2', 'CH4', 'C2H2', 'C2H4', 'C2H6', 'Fault'], dtype=object)

In [62]:
discrete_columns = ['H2', 'CH4', 'C2H2', 'C2H4', 'C2H6', 'Fault']

In [63]:
ctgan = CTGANSynthesizer(epochs=200)
ctgan.fit(data1, discrete_columns)

In [64]:
samples1 = ctgan.sample(5000)
samples1.head()
print(samples1.shape)

(5000, 6)


In [65]:
evaluate(samples1, df1)

0.7389867445145314

It is observed that there is a slight improvement in evaluation if we dont use label encoding

Saving the samples generated into CSV file

In [66]:
samples1.to_csv("/content/gdrive/My Drive/Colab Notebooks/Synthetic_Data/CTGANSynth_200.csv", index = False)

In [88]:
ctgan1 = CTGANSynthesizer(epochs=500)
ctgan1.fit(data1, discrete_columns)
samples2 = ctgan.sample(5000)
print(evaluate(samples2, df1))
samples2.to_csv("/content/gdrive/My Drive/Colab Notebooks/Synthetic_Data/CTGANSynth_500.csv", index = False)

0.7386453645530792


## Create new samples using CTGAN

In [67]:
data_aug1 = df1.copy()

In [68]:
data_aug1.head()

,H2,CH4,C2H2,C2H4,C2H6,Fault
0,80.00,10.00,0.00,4.00,1.50,NF
1,46.13,11.57,0.63,33.14,8.52,NF
2,33.66,2.97,2.48,33.17,27.72,NF
3,143.80,21.53,0.20,8.21,2.71,NF
4,170.00,300.00,8.90,580.00,44.00,T3


In [81]:
model1 = CTGAN()

In [82]:
model1.fit(data_aug1)

In [83]:
augmented_data1 = model1.sample(5000)
augmented_data1.head()

,H2,CH4,C2H2,C2H4,C2H6,Fault
0,0.000,1284.41,2677.818,279.35,0.00,T3
1,0.000,0.00,0.000,0.00,0.00,D2
2,6758.995,0.00,4809.689,558.44,628.01,T3
3,0.000,4320.18,81808.058,0.00,0.00,D1
4,3803.956,0.00,0.000,0.00,0.00,D1


In [84]:
evaluate(augmented_data1, df1)

0.5603327915493915

It is observed taht CTGAN gives better evaluation without label encoding

## Hyperparameter Tuning of CTGAN

It is observed that input data without Label Encoding gies better accuracy

In [85]:
filename = "Consolidated_Actual_Fault_Data.csv"
df2 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/" + filename)
display(df2)

,H2,CH4,C2H2,C2H4,C2H6,Fault
0,80.00,10.00,0.00,4.00,1.50,NF
1,46.13,11.57,0.63,33.14,8.52,NF
2,33.66,2.97,2.48,33.17,27.72,NF
3,143.80,21.53,0.20,8.21,2.71,NF
4,170.00,300.00,8.90,580.00,44.00,T3
...,...,...,...,...,...,...
468,165.62,240.95,61.32,514.53,13.53,T3
469,164.00,244.00,103.00,497.00,8.30,T3
470,135.65,278.53,58.86,492.00,2.95,T3
471,135.88,362.42,125.22,826.65,3.74,T3


In [87]:
epochs = [500, 750, 1000]
models = []
for epoch in epochs:
  model3 = CTGAN(
      epochs=epoch,
      batch_size=30,
      log_frequency=False,
      generator_dim=(256, 256, 256),
      discriminator_dim=(256, 256, 256),
      verbose=True
      )
  models.append(model3)
co = 1
for model4 in models:
  model4.fit(data_aug1)
  augmented_data2 = model4.sample(5000)
  print(evaluate(augmented_data2, df2))
  augmented_data2.to_csv("/content/gdrive/My Drive/Colab Notebooks/Synthetic_Data/CTGANSynth_Hyp_" + str(co) + ".csv", index = False)
  co += 1

Epoch 1, Loss G:  1.7188,Loss D: -0.0194
Epoch 2, Loss G:  1.6883,Loss D: -0.1823
Epoch 3, Loss G:  1.2884,Loss D: -0.3011
Epoch 4, Loss G:  0.6065,Loss D: -0.0324
Epoch 5, Loss G: -0.1304,Loss D:  0.3103
Epoch 6, Loss G: -0.6913,Loss D: -0.1898
Epoch 7, Loss G: -0.3509,Loss D: -0.1639
Epoch 8, Loss G: -0.8389,Loss D:  0.1498
Epoch 9, Loss G: -0.6702,Loss D:  0.3745
Epoch 10, Loss G:  0.0287,Loss D: -0.1918
Epoch 11, Loss G: -0.7756,Loss D:  0.4646
Epoch 12, Loss G: -0.3712,Loss D:  0.5536
Epoch 13, Loss G: -0.5831,Loss D:  0.6901
Epoch 14, Loss G: -0.2835,Loss D: -0.2076
Epoch 15, Loss G: -0.2977,Loss D:  0.3647
Epoch 16, Loss G: -0.7991,Loss D:  0.4086
Epoch 17, Loss G: -0.4963,Loss D: -0.0695
Epoch 18, Loss G: -0.0536,Loss D:  0.2411
Epoch 19, Loss G: -0.7176,Loss D:  0.6528
Epoch 20, Loss G: -0.2053,Loss D:  0.6875
Epoch 21, Loss G: -0.3159,Loss D:  0.2584
Epoch 22, Loss G: -0.6826,Loss D:  0.1438
Epoch 23, Loss G: -0.0634,Loss D:  0.0503
Epoch 24, Loss G: -0.3140,Loss D:  0.4126
E